In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import unidecode

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Criação de DF para filtragem dos selecionados

In [3]:
df = pd.read_parquet('CAD_UNIC_MAI_DT_23_04_05_23.parquet')

In [4]:
df_renda_pobreza_89 = df[df['d.vlr_renda_media_fam'] <= 89]

In [6]:
df_analise_2 = df_renda_pobreza_89[['d.cod_familiar_fam', 'd.cod_material_piso_fam', 'd.cod_material_domic_fam', 'd.cod_banheiro_domic_fam', 'p.cod_sexo_pessoa', 'p.cod_deficiencia_memb', 'p.cod_raca_cor_pessoa', 'p.fx_idade', 'd.qtd_comodos_domic_fam']]

In [7]:
df_analise_final_89 = df_analise_2.groupby('d.cod_familiar_fam')\
    .agg({'d.cod_material_domic_fam': [lambda x: 2 in x.values, lambda x: 1 in x.values], # 2 sem e 1 com
         'd.cod_material_piso_fam': [lambda x: 2 in x.values, lambda x: 1 in x.values, lambda x: 5 in x.values],# 2-Cimento|1-Terra|3-ceramica e tals
         'd.cod_banheiro_domic_fam': lambda x: 1 in x.values, #1-Sim|2-Não
         'p.cod_raca_cor_pessoa': [lambda x: 2 in x.values or 4 in x.values],
         'p.cod_sexo_pessoa': lambda x: (2 == x.values).all(),
         'p.fx_idade': lambda x: 0 in x.values,
         'p.cod_deficiencia_memb': lambda x: 1 in x.values,
         'd.qtd_comodos_domic_fam': [lambda x: 1 in x.values or 2 in x.values, 
                                     lambda x: 3 in x.values or 4 in x.values or 5 in x.values]})\
    .rename(columns = {'d.cod_material_domic_fam': 'Alvenaria'}).reset_index()

In [8]:
df_analise_final_89

d.cod_familiar_fam  Alvenaria            d.cod_material_piso_fam                       d.cod_banheiro_domic_fam p.cod_raca_cor_pessoa p.cod_sexo_pessoa p.fx_idade p.cod_deficiencia_memb d.qtd_comodos_domic_fam           
                         <lambda_0> <lambda_1>              <lambda_0> <lambda_1> <lambda_2>                 <lambda>              <lambda>          <lambda>   <lambda>               <lambda>              <lambda_0> <lambda_1>
0            00030988500      False       True                   False      False       True                     True                  True             False       True                  False                   False       True
1            00032367570      False      False                    True      False      False                     True                  True             False       True                  False                   False       True
2            00068828039       True      False                    True      False      False                     True                  True             False      False                  False                   False       True
3            00103947965      False       True                   False      False       True                     True                  True             False      False                  False                   False       True
4            00106576577       True      False                   False      False       True                     True                  True              True      False                   True                   False       True
...                  ...        ...        ...                     ...        ...        ...                      ...                   ...               ...        ...                    ...                     ...        ...
77393        07850906144      False       True                   False      False       True                     True                  True              True      False                  False                   False       True
77394        07851150990      False       True                    True      False      False                     True                  True             False      False                  False                    True      False
77395        07851175046      False       True                    True      False      False                     True                  True              True      False                  False                   False       True
77396        07851465478      False      False                   False      False      False                     True                 False             False      False                  False                    True      False
77397        07851607280       True      False                    True      False      False                     True                 False             False      False                  False                    True      False

[77398 rows x 13 columns]

In [9]:
df_analise_final_89.columns = ['_'.join(col) for col in df_analise_final_89.columns]

In [10]:
df_analise_final_89.columns=['d.cod_familiar_fam', 'sem_revestimento', 'com_revestimento',
                      'piso_cimento', 'piso_terra', 'piso_ceramica_afins', 'banheiro',
                      'pretos_ou_pardos', 'familia_de_mulheres' ,'primeira_infancia', 'pcd', 'comodos_0-2',
                     'comodos_3-5']

In [11]:
df_analise_final_89 = df_analise_final_89.replace(True, 1)
df_analise_final_89 = df_analise_final_89.replace(False, 0)

In [12]:
df_analise_final_89

,d.cod_familiar_fam,sem_revestimento,com_revestimento,piso_cimento,piso_terra,piso_ceramica_afins,banheiro,pretos_ou_pardos,familia_de_mulheres,primeira_infancia,pcd,comodos_0-2,comodos_3-5
0,00030988500,0,1,0,0,1,1,1,0,1,0,0,1
1,00032367570,0,0,1,0,0,1,1,0,1,0,0,1
2,00068828039,1,0,1,0,0,1,1,0,0,0,0,1
3,00103947965,0,1,0,0,1,1,1,0,0,0,0,1
4,00106576577,1,0,0,0,1,1,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77393,07850906144,0,1,0,0,1,1,1,1,0,0,0,1
77394,07851150990,0,1,1,0,0,1,1,0,0,0,1,0
77395,07851175046,0,1,1,0,0,1,1,1,0,0,0,1
77396,07851465478,0,0,0,0,0,1,0,0,0,0,1,0


## Filtros escolhidos e nova montagem de DF de domicílios selecionados

#### Recorte 1

In [15]:
df_filtro1 = df_analise_final_89[(df_analise_final_89['banheiro']==0) & (df_analise_final_89['primeira_infancia']==1)]
df_filtro2 = df_analise_final_89[(df_analise_final_89['banheiro']==0) & (df_analise_final_89['pcd']==1)]
df_filtro3 = df_analise_final_89[(df_analise_final_89['sem_revestimento']==1) & (df_analise_final_89['piso_terra']==1) & (df_analise_final_89['primeira_infancia']==1)]
df_filtro4 = df_analise_final_89[(df_analise_final_89['sem_revestimento']==1) & (df_analise_final_89['piso_terra']==1) & (df_analise_final_89['pcd']==1)]
df_filtro5 = df_analise_final_89[(df_analise_final_89['sem_revestimento']==1) & (df_analise_final_89['piso_cimento']==1) & (df_analise_final_89['primeira_infancia']==1)]
df_filtro6 = df_analise_final_89[(df_analise_final_89['sem_revestimento']==1) & (df_analise_final_89['piso_cimento']==1) & (df_analise_final_89['pcd']==1)]

In [16]:
data=[df_filtro1, df_filtro2, df_filtro3, df_filtro4, df_filtro5, df_filtro6]
df_filtro_final = pd.concat(data)

In [17]:
df_filtro_final = df_filtro_final.drop_duplicates(keep='first')

In [18]:
df_filtro_final

,d.cod_familiar_fam,sem_revestimento,com_revestimento,piso_cimento,piso_terra,piso_ceramica_afins,banheiro,pretos_ou_pardos,familia_de_mulheres,primeira_infancia,pcd,comodos_0-2,comodos_3-5
334,00549540610,1,0,1,0,0,0,1,0,1,0,1,0
856,00743168933,0,1,0,0,1,0,1,0,1,0,0,0
3565,01326803000,0,1,1,0,0,0,1,0,1,0,1,0
3929,01327198860,0,0,0,0,0,0,1,0,1,0,0,0
4198,01338172107,1,0,1,0,0,0,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77100,07805445796,1,0,1,0,0,1,1,1,0,1,1,0
77279,07835226065,1,0,1,0,0,1,1,0,0,1,1,0
77288,07836795136,1,0,1,0,0,1,0,0,0,1,1,0
77318,07841832605,1,0,1,0,0,1,1,0,0,1,1,0


In [19]:
df_filtro_1 = df_analise_final_89[(df_analise_final_89['banheiro']==0) & (df_analise_final_89['pcd']==1) & (df_analise_final_89['familia_de_mulheres']==1)]
df_filtro_2 = df_analise_final_89[(df_analise_final_89['banheiro']==0) & (df_analise_final_89['pcd']==1) & (df_analise_final_89['pretos_ou_pardos']==1)].sample(176)
df_filtro3 = df_analise_final_89[(df_analise_final_89['banheiro']==0) & (df_analise_final_89['pcd']==1) & (df_analise_final_89['primeira_infancia']==1)]
df_filtro_4 = df_analise_final_89[(df_analise_final_89['banheiro']==0) & (df_analise_final_89['pcd']==1)].sample(35)

In [20]:
data=[df_filtro_1, df_filtro_2, df_filtro3, df_filtro_4]
df_filtro1e2 = pd.concat(data)

In [21]:
df_filtro1e2['TP-04'] = 0
df_filtro1e2['TP-05'] = 0
df_filtro1e2['TP-06'] = 1
df_filtro1e2['TP-07'] = 0
df_filtro1e2['TC-09'] = 1
df_filtro1e2['TC-10'] = 0
df_filtro1e2['TC-12'] = 0

In [22]:
df_filtro_parte_1 = df_filtro1e2.drop_duplicates(keep='first')

In [23]:
filtro_piso_cimento_89 = df_analise_final_89[(df_analise_final_89['piso_cimento']==1)]
filtro_piso_terra_89 = df_analise_final_89[(df_analise_final_89['piso_terra']==1)]

In [24]:
filtro_negros = filtro_piso_cimento_89[(filtro_piso_cimento_89['pretos_ou_pardos']==1)].sample(10000)

In [25]:
filtro_negros_e_mulheres = filtro_piso_cimento_89[(filtro_piso_cimento_89['pretos_ou_pardos']==1) & (filtro_piso_cimento_89['familia_de_mulheres']==1)]

In [26]:
filtro_mulheres = filtro_piso_cimento_89[(filtro_piso_cimento_89['familia_de_mulheres']==1)]

In [27]:
data=[filtro_negros_e_mulheres, filtro_piso_terra_89, filtro_mulheres, filtro_negros]
filtro_piso = pd.concat(data)

In [28]:
df_filtro_parte_2 = filtro_piso.drop_duplicates(keep='first')

In [29]:
df_filtro_parte_2['TP-04'] = 0
df_filtro_parte_2['TP-05'] = 0
df_filtro_parte_2['TP-06'] = 1
df_filtro_parte_2['TP-07'] = 0
df_filtro_parte_2['TC-09'] = 0
df_filtro_parte_2['TC-10'] = 0
df_filtro_parte_2['TC-12'] = 0

C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_3624\8323674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtro_parte_2['TP-04'] = 0
C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_3624\8323674.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtro_parte_2['TP-05'] = 0
C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_3624\8323674.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [30]:
data=[df_filtro_parte_1, df_filtro_parte_2]
filtro_parte_final = pd.concat(data)

In [31]:
df_filtro_parte_3 = filtro_parte_final.drop_duplicates(keep='first')

In [32]:
df_sample_1 = df_filtro_parte_3[(df_filtro_parte_3['TP-06']==1)].sample(2565, random_state=1)

In [33]:
df_sample_1['TC-12'] = 1

In [34]:
data=[df_filtro_parte_3, df_sample_1]
filtro_parte_3 = pd.concat(data)

In [35]:
df_filtro_parte_3 = filtro_parte_3.drop_duplicates(subset=['d.cod_familiar_fam'], keep='last')

In [36]:
df_sample_2 = df_filtro_parte_3[(df_filtro_parte_3['TC-12']==1)].sample(376, random_state=2)

In [37]:
df_sample_2['TP-04'] = 1
data=[df_filtro_parte_3, df_sample_2]
filtro_parte_3 = pd.concat(data)

In [38]:
df_filtro_parte_3 = filtro_parte_3.drop_duplicates(subset=['d.cod_familiar_fam'], keep='last')

In [40]:
df_filtro_parte_3_copia = df_filtro_parte_3.copy()

In [41]:
df_filtro_parte_3_copia['sem_banheiro'] = df_filtro_parte_3_copia['banheiro']

In [42]:
dict0e1={
    0: 1,
    1: 0
}

df_filtro_parte_3_copia['sem_banheiro'] = df_filtro_parte_3_copia['sem_banheiro'].map(dict0e1)

#### Recorte 2

In [43]:
filtro_banheiro_pcd_89 = df_analise_final_89[(df_analise_final_89['banheiro']==1) & (df_analise_final_89['pcd']==1)]
filtro_banheiro_mulheres_89 = df_analise_final_89[(df_analise_final_89['banheiro']==1) & (df_analise_final_89['familia_de_mulheres']==1)].sample(17000)
filtro_filtro = df_analise_final_89[(df_analise_final_89['banheiro']==1) & (df_analise_final_89['primeira_infancia']==1)].sample(5554)

In [44]:
data=[filtro_banheiro_pcd_89, filtro_banheiro_mulheres_89, filtro_filtro]
df_filtro_filtro = pd.concat(data)

In [46]:
data=[filtro_banheiro_pcd_89, filtro_banheiro_mulheres_89, df_filtro_filtro]
df_filtro_parte_4 = pd.concat(data)

In [47]:
df_filtro_parte_4 = df_filtro_parte_4.drop_duplicates(keep='first')

In [48]:
df_filtro_parte_4['TP-04'] = 0
df_filtro_parte_4['TP-05'] = 0
df_filtro_parte_4['TP-06'] = 0
df_filtro_parte_4['TP-07'] = 0
df_filtro_parte_4['TC-09'] = 0
df_filtro_parte_4['TC-10'] = 1
df_filtro_parte_4['TC-12'] = 0

In [49]:
df_sample_3 = df_filtro_parte_4.copy()

In [50]:
data=[df_filtro_parte_3, df_sample_3]
df_filtro_parte_3_e_4 = pd.concat(data)

In [51]:
df_filtro_parte_5 = df_filtro_parte_3_e_4.drop_duplicates(subset=['d.cod_familiar_fam'], keep='first')

In [52]:
df_sample_4 = df_filtro_parte_5[(df_filtro_parte_5['TC-10'] == 1) & (df_filtro_parte_5['sem_revestimento'] == 1)]

In [53]:
df_sample_4['TP-07'] = 1

C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_3624\1898523591.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample_4['TP-07'] = 1


In [54]:
data=[df_filtro_parte_5, df_sample_4]
filtro_parte_5 = pd.concat(data)

In [55]:
df_filtro_parte_5 = filtro_parte_5.drop_duplicates(subset=['d.cod_familiar_fam'], keep='last')

In [56]:
df_sample_5 = df_filtro_parte_5[df_filtro_parte_5['TP-07']==1]

In [57]:
df_sample_5['TP-04'] = 1
df_sample_5['TP-05'] = 1

C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_3624\1681526806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample_5['TP-04'] = 1
C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_3624\1681526806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample_5['TP-05'] = 1


In [58]:
data=[df_filtro_parte_5, df_sample_5]
filtro_parte_6 = pd.concat(data)

In [59]:
df_filtro_parte_5 = filtro_parte_6.drop_duplicates(subset=['d.cod_familiar_fam'], keep='last')

In [60]:
df_sample_6 = df_filtro_parte_5[(df_filtro_parte_5['TP-06']==1) & (df_filtro_parte_5['TC-09']==0) & (df_filtro_parte_5['TP-07']==0)].sample(6696, random_state=4)

In [61]:
df_sample_6['TP-07'] = 1

In [62]:
data=[df_filtro_parte_5, df_sample_6]
filtro_parte_7 = pd.concat(data)

In [63]:
df_filtro_parte_6 = filtro_parte_7.drop_duplicates(subset=['d.cod_familiar_fam'], keep='last')

In [64]:
df_sample_7 = df_sample_6.sample(780, random_state=5)

In [65]:
df_sample_7['TP-04'] = 1
df_sample_7['TP-05'] = 1

In [66]:
data=[df_filtro_parte_5, df_sample_7]
filtro_parte_8 = pd.concat(data)

In [67]:
df_filtro_parte_7 = filtro_parte_8.drop_duplicates(subset=['d.cod_familiar_fam'], keep='last')

In [68]:
df_filtro_parte_7['sem_banheiro'] = df_filtro_parte_7['banheiro']

C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_3624\4176658757.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtro_parte_7['sem_banheiro'] = df_filtro_parte_7['banheiro']


In [69]:
dict0e1={
    0: 1,
    1: 0
}

df_filtro_parte_7['sem_banheiro'] = df_filtro_parte_7['sem_banheiro'].map(dict0e1)

C:\Users\alexandre.goncalves\AppData\Local\Temp\ipykernel_3624\1651994130.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtro_parte_7['sem_banheiro'] = df_filtro_parte_7['sem_banheiro'].map(dict0e1)


In [70]:
df_filtro_parte_7 = df_filtro_parte_7[['d.cod_familiar_fam',
 'sem_revestimento',
 'com_revestimento',
 'piso_cimento',
 'piso_terra',
 'piso_ceramica_afins',
 'banheiro',
 'sem_banheiro',
 'pretos_ou_pardos',
 'familia_de_mulheres',
 'primeira_infancia',
 'pcd',
 'comodos_0-2',
 'comodos_3-5',
 'TP-04',
 'TP-05',
 'TP-06',
 'TP-07',
 'TC-09',
 'TC-10',
 'TC-12']]

In [72]:
df_pessoas_selecionadas = df_filtro_parte_7[~df_filtro_parte_7.index.isin(df_filtro_parte_3_copia.index)]